<center>
  <h2>Trabajo Laboratorio de Implementación III - MCD Virtual Cohorte 2022</h2>
  <h3>Forecasting para una Empresa de Consumo Masivo</h3>
  <h4>Evaluación</h4>
</center>

In [1]:
# Importación librerias

# !pip install awswrangler
# !pip install boto3
import boto3
import awswrangler as wr
import pandas as pd
import numpy as np
from datetime import datetime
from dateutil.relativedelta import relativedelta

In [2]:
# Definición fechas

date_start=datetime.strptime('2017-01-01', '%Y-%m-%d')
date_end=datetime.strptime('2019-12-01', '%Y-%m-%d')
forecast_length = 2
date_future=date_end+relativedelta(months=forecast_length)
years=range(2018, 2020)

print(f"Fecha inicio historia {date_start}")
print(f"Fecha fin historia {date_end}")
print(f"Fecha futura (related time series) {date_future}")

# Versión modelo
data_version_cluster1 = '4_c1'
data_version_cluster2 = '4_c2'
data_version_cluster3 = '4_c3'
grouped_columns = ["product_id"]
grouped_cols_name = "product"

# Parametros backtest
dataset_frequency='M'
quantiles=["mean"]

Fecha inicio historia 2017-01-01 00:00:00
Fecha fin historia 2019-12-01 00:00:00
Fecha futura (related time series) 2020-02-01 00:00:00


In [3]:
# Conexiones AWS

account_id=boto3.client('sts').get_caller_identity().get('Account')
session = boto3.Session() 
forecast = session.client(service_name='forecast')
region = forecast.meta.region_name
forecastquery = session.client(service_name='forecastquery')
role = f'arn:aws:iam::{account_id}:role/ForecastRole'

In [10]:
# Buckets de datos

# CLUSTER 1
target_s3_path_c1=f's3://datasets-forecast-account1/modelo-{data_version_cluster1}-forecast-{grouped_cols_name}/target/df_ventas_{grouped_cols_name}.csv'
items_s3_path_c1=f's3://datasets-forecast-account1/modelo-{data_version_cluster1}-forecast-{grouped_cols_name}/product/df_products.csv'
related_s3_path_c1=f's3://datasets-forecast-account1/modelo-{data_version_cluster1}-forecast-{grouped_cols_name}/related/df_related_{grouped_cols_name}.csv'

# CLUSTER 2
target_s3_path_c2=f's3://datasets-forecast-account1/modelo-{data_version_cluster2}-forecast-{grouped_cols_name}/target/df_ventas_{grouped_cols_name}.csv'
items_s3_path_c2=f's3://datasets-forecast-account1/modelo-{data_version_cluster2}-forecast-{grouped_cols_name}/product/df_products.csv'
related_s3_path_c2=f's3://datasets-forecast-account1/modelo-{data_version_cluster2}-forecast-{grouped_cols_name}/related/df_related_{grouped_cols_name}.csv'

# CLUSTER 3
target_s3_path_c3=f's3://datasets-forecast-account1/modelo-{data_version_cluster3}-forecast-{grouped_cols_name}/target/df_ventas_{grouped_cols_name}.csv'
items_s3_path_c3=f's3://datasets-forecast-account1/modelo-{data_version_cluster3}-forecast-{grouped_cols_name}/product/df_products.csv'
related_s3_path_c3=f's3://datasets-forecast-account1/modelo-{data_version_cluster3}-forecast-{grouped_cols_name}/related/df_related_{grouped_cols_name}.csv'

# Outputs
backtest_export_path_c1=f's3://datasets-forecast-account1/modelo-{data_version_cluster1}-forecast-{grouped_cols_name}/output/'
backtest_export_path_c2=f's3://datasets-forecast-account1/modelo-{data_version_cluster2}-forecast-{grouped_cols_name}/output/'
backtest_export_path_c3=f's3://datasets-forecast-account1/modelo-{data_version_cluster3}-forecast-{grouped_cols_name}/output/'

In [5]:
# Formar ARNs

# CLUSTER 1
dataset_group_arn_c1=f'arn:aws:forecast:us-east-1:{account_id}:dataset-group/modelo_{data_version_cluster1}_{grouped_cols_name}'
automl_predictor_arn_c1 = f'arn:aws:forecast:us-east-1:{account_id}:predictor/modelo_{data_version_cluster1}_predictor_01J02Q23DBHN6P2C8QM61MAY3F'

# CLUSTER 2
dataset_group_arn_c2=f'arn:aws:forecast:us-east-1:{account_id}:dataset-group/modelo_{data_version_cluster2}_{grouped_cols_name}'
automl_predictor_arn_c2 = f'arn:aws:forecast:us-east-1:{account_id}:predictor/modelo_{data_version_cluster2}_predictor_01J02Q4FXFQADEHVVGBN5ETE7C'

# CLUSTER 3
dataset_group_arn_c3=f'arn:aws:forecast:us-east-1:{account_id}:dataset-group/modelo_{data_version_cluster3}_{grouped_cols_name}'
automl_predictor_arn_c3 = f'arn:aws:forecast:us-east-1:{account_id}:predictor/modelo_{data_version_cluster3}_predictor_01J02Q5EW0HRWSQEVHF4M5B8WP'

In [6]:
# Verificar comunicación con Amazon Forecast
assert forecast.list_predictors()

### Backtest export job

In [11]:
# Nombre del projecto y dataset group
# CLUSTER 1
backtest_export_job_name_c1 = f"modelo_{data_version_cluster1}_predictor_export"
response_c1 = forecast.create_predictor_backtest_export_job(
    PredictorBacktestExportJobName=backtest_export_job_name_c1,
    PredictorArn=automl_predictor_arn_c1,
    Destination={
        'S3Config': {
            'Path': backtest_export_path_c1,
            'RoleArn': role
        }
    },
    Format='CSV'
)
backtest_export_job_arn_c1 = response_c1['PredictorBacktestExportJobArn']

In [22]:
print(f"Waiting for Backtest export job with ARN {backtest_export_job_arn_c1} to be completed. \nCurrent Status:")
forecast.describe_predictor_backtest_export_job(PredictorBacktestExportJobArn=
                backtest_export_job_arn_c1)['Status']

Waiting for Backtest export job with ARN arn:aws:forecast:us-east-1:935182042847:predictor-backtest-export-job/modelo_4_c1_predictor/modelo_4_c1_predictor_export to be completed. 
Current Status:


'ACTIVE'

In [13]:
# Nombre del projecto y dataset group
# CLUSTER 2
backtest_export_job_name_c2 = f"modelo_{data_version_cluster2}_predictor_export"
response_c2 = forecast.create_predictor_backtest_export_job(
    PredictorBacktestExportJobName=backtest_export_job_name_c2,
    PredictorArn=automl_predictor_arn_c2,
    Destination={
        'S3Config': {
            'Path': backtest_export_path_c2,
            'RoleArn': role
        }
    },
    Format='CSV'
)
backtest_export_job_arn_c2 = response_c2['PredictorBacktestExportJobArn']

In [21]:
print(f"Waiting for Backtest export job with ARN {backtest_export_job_arn_c2} to be completed. \nCurrent Status:")
forecast.describe_predictor_backtest_export_job(PredictorBacktestExportJobArn=
                backtest_export_job_arn_c2)['Status']

Waiting for Backtest export job with ARN arn:aws:forecast:us-east-1:935182042847:predictor-backtest-export-job/modelo_4_c2_predictor/modelo_4_c2_predictor_export to be completed. 
Current Status:


'ACTIVE'

In [15]:
# Nombre del projecto y dataset group
# CLUSTER 3
backtest_export_job_name_c3 = f"modelo_{data_version_cluster3}_predictor_export"
response_c3 = forecast.create_predictor_backtest_export_job(
    PredictorBacktestExportJobName=backtest_export_job_name_c3,
    PredictorArn=automl_predictor_arn_c3,
    Destination={
        'S3Config': {
            'Path': backtest_export_path_c3,
            'RoleArn': role
        }
    },
    Format='CSV'
)
backtest_export_job_arn_c3 = response_c3['PredictorBacktestExportJobArn']

In [26]:
print(f"Waiting for Backtest export job with ARN {backtest_export_job_arn_c3} to be completed. \nCurrent Status:")
forecast.describe_predictor_backtest_export_job(PredictorBacktestExportJobArn=
                backtest_export_job_arn_c3)['Status']

Waiting for Backtest export job with ARN arn:aws:forecast:us-east-1:935182042847:predictor-backtest-export-job/modelo_4_c3_predictor/modelo_4_c3_predictor_export to be completed. 
Current Status:


'ACTIVE'

### Evaluate

In [28]:
df_forecasts_c1=wr.s3.read_csv(backtest_export_path_c1 + 'forecasted-values/')
print("----------C1-----------")
print(df_forecasts_c1.timestamp.min(), df_forecasts_c1.timestamp.max())
print(len(df_forecasts_c1.item_id.drop_duplicates()))
print(df_forecasts_c1.head(1))

df_forecasts_c2=wr.s3.read_csv(backtest_export_path_c2 + 'forecasted-values/')
print("----------C2-----------")
print(df_forecasts_c2.timestamp.min(), df_forecasts_c2.timestamp.max())
print(len(df_forecasts_c2.item_id.drop_duplicates()))
print(df_forecasts_c2.head(1))

df_forecasts_c3=wr.s3.read_csv(backtest_export_path_c3 + 'forecasted-values/')
print("----------C3-----------")
print(df_forecasts_c3.timestamp.min(), df_forecasts_c3.timestamp.max())
print(len(df_forecasts_c3.item_id.drop_duplicates()))
print(df_forecasts_c3.head(1))

----------C1-----------
2019-11-01T00:00:00Z 2019-12-01T00:00:00Z
245
   item_id             timestamp  target_value backtestwindow_start_time   
0    20158  2019-11-01T00:00:00Z        67.906       2019-11-01T00:00:00  \

  backtestwindow_end_time       mean  
0     2019-12-01T00:00:00  71.620168  
----------C2-----------
2019-11-01T00:00:00Z 2019-12-01T00:00:00Z
394
   item_id             timestamp  target_value backtestwindow_start_time   
0    20219  2019-11-01T00:00:00Z       30.5133       2019-11-01T00:00:00  \

  backtestwindow_end_time       mean  
0     2019-12-01T00:00:00  19.341341  
----------C3-----------
2019-11-01T00:00:00Z 2019-12-01T00:00:00Z
141
   item_id             timestamp  target_value backtestwindow_start_time   
0    20569  2019-11-01T00:00:00Z       12.3187       2019-11-01T00:00:00  \

  backtestwindow_end_time      mean  
0     2019-12-01T00:00:00  8.975171  


In [29]:
df_forecasts = pd.concat([df_forecasts_c1, df_forecasts_c2, df_forecasts_c3])
df_forecasts = df_forecasts.sort_values(by = ["timestamp", "item_id"])

In [30]:
len(df_forecasts) == len(df_forecasts_c1) + len(df_forecasts_c2) + len(df_forecasts_c3)

True

In [31]:
df_forecasts['periodo']=pd.to_datetime(df_forecasts['timestamp'], format='%Y-%m-%dT%H:%M:%SZ').dt.date
df_forecasts['product_id']=df_forecasts['item_id']
df_forecasts = df_forecasts.drop(columns = {"item_id", "target_value", "backtestwindow_start_time", "backtestwindow_end_time", "timestamp"})
for q in quantiles:
    df_forecasts[q] = df_forecasts[q].astype(float)
df_forecasts.head()
df_forecasts = df_forecasts[["periodo"] + grouped_columns + quantiles]
df_forecasts['product_id']=df_forecasts['product_id'].astype(str)
df_forecasts['periodo']=pd.to_datetime(df_forecasts['periodo'])
df_forecasts.head(5)

,periodo,product_id,mean
262,2019-11-01,20001,1537.775634
488,2019-11-01,20002,1337.863282
146,2019-11-01,20003,1105.423166
360,2019-11-01,20004,913.937149
122,2019-11-01,20005,713.956175


- Backtest timing

In [32]:
print('Start backtest: ', df_forecasts['periodo'].astype(str).min())
print('End backtest: ', df_forecasts['periodo'].astype(str).max())

Start backtest:  2019-11-01
End backtest:  2019-12-01


In [33]:
date_min_forecast = df_forecasts['periodo'].astype(str).min()
date_min_forecast = pd.to_datetime(date_min_forecast)

- Backtest items check

In [34]:
len(df_forecasts.product_id.unique())

780

- Read historic sales data

In [37]:
# CLUSTER1 
df_ventas_mensuales_c1 = wr.s3.read_csv(target_s3_path_c1, header=None, index_col=False)
column_names = ["periodo"] + grouped_columns + ["venta"]
df_ventas_mensuales_c1.columns = column_names
print("----------C1-----------")
print(df_ventas_mensuales_c1.periodo.min(), df_ventas_mensuales_c1.periodo.max())
print(len(df_ventas_mensuales_c1.product_id.drop_duplicates()))
print(df_ventas_mensuales_c1.head(1))

# CLUSTER2
df_ventas_mensuales_c2 = wr.s3.read_csv(target_s3_path_c2, header=None, index_col=False)
column_names = ["periodo"] + grouped_columns + ["venta"]
df_ventas_mensuales_c2.columns = column_names
print("----------C2-----------")
print(df_ventas_mensuales_c2.periodo.min(), df_ventas_mensuales_c2.periodo.max())
print(len(df_ventas_mensuales_c2.product_id.drop_duplicates()))
print(df_ventas_mensuales_c2.head(1))

# CLUSTER3
df_ventas_mensuales_c3 = wr.s3.read_csv(target_s3_path_c3, header=None, index_col=False)
column_names = ["periodo"] + grouped_columns + ["venta"]
df_ventas_mensuales_c3.columns = column_names
print("----------C3-----------")
print(df_ventas_mensuales_c3.periodo.min(), df_ventas_mensuales_c3.periodo.max())
print(len(df_ventas_mensuales_c3.product_id.drop_duplicates()))
print(df_ventas_mensuales_c3.head(1))

----------C1-----------
2017-01-01 2019-12-01
245
      periodo  product_id      venta
0  2017-01-01       20001  934.77222
----------C2-----------
2017-01-01 2019-12-01
394
      periodo  product_id      venta
0  2017-01-01       20006  528.40996
----------C3-----------
2017-01-01 2019-12-01
141
      periodo  product_id  venta
0  2017-01-01       20032    0.0


In [38]:
df_ventas_mensuales = pd.concat([df_ventas_mensuales_c1, df_ventas_mensuales_c2, df_ventas_mensuales_c3])
df_ventas_mensuales = df_ventas_mensuales.sort_values(by = ["periodo", "product_id"])

In [39]:
len(df_ventas_mensuales) == len(df_ventas_mensuales_c1) + len(df_ventas_mensuales_c2) + len(df_ventas_mensuales_c3)

True

In [40]:
df_ventas_mensuales['periodo'] = pd.to_datetime(df_ventas_mensuales['periodo'])
df_ventas_mensuales['product_id']=df_ventas_mensuales['product_id'].astype(str)
df_ventas_mensuales.head(5)

,periodo,product_id,venta
0,2017-01-01,20001,934.77222
1,2017-01-01,20002,550.15707
2,2017-01-01,20003,1063.45835
3,2017-01-01,20004,555.91614
4,2017-01-01,20005,494.27011


In [41]:
df_ventas_mensuales.dtypes

periodo       datetime64[ns]
product_id            object
venta                float64
dtype: object

In [42]:
len(df_ventas_mensuales.product_id.unique())

780

In [43]:
print('Start historical sales: ', df_ventas_mensuales['periodo'].min())
print('End historical sales: ', df_ventas_mensuales['periodo'].max())

Start historical sales:  2017-01-01 00:00:00
End historical sales:  2019-12-01 00:00:00


- Unión info histórica y forecast

In [44]:
df_forecasts.dtypes

periodo       datetime64[ns]
product_id            object
mean                 float64
dtype: object

In [45]:
df_ventas_mensuales.dtypes

periodo       datetime64[ns]
product_id            object
venta                float64
dtype: object

In [46]:
# Para los meses en los que coexiste venta real con forecast se realiza un merge outer
df_forecasts_historic=pd.merge(df_forecasts[df_forecasts['periodo']>=date_min_forecast], df_ventas_mensuales[df_ventas_mensuales['periodo']>=date_min_forecast], how='outer', on=['periodo'] + grouped_columns)
df_forecasts_historic=df_forecasts_historic[['periodo'] + grouped_columns + ['venta'] + quantiles]
# Para los meses históricos, previos a la proyección de forecast, se realiza un concat
df_forecasts_historic = pd.concat([df_ventas_mensuales[df_ventas_mensuales['periodo']<date_min_forecast], df_forecasts_historic])
df_forecasts_historic.head(5)

,periodo,product_id,venta,mean
0,2017-01-01,20001,934.77222,NaN
1,2017-01-01,20002,550.15707,NaN
2,2017-01-01,20003,1063.45835,NaN
3,2017-01-01,20004,555.91614,NaN
4,2017-01-01,20005,494.27011,NaN


In [48]:
df_forecasts_historic_check = df_forecasts_historic.copy()
df_ventas_mensuales_check = df_ventas_mensuales.copy()
len(df_forecasts_historic_check) == len(df_forecasts[df_forecasts['periodo']>=date_min_forecast]) + len(df_ventas_mensuales_check[df_ventas_mensuales_check['periodo']<date_min_forecast])

True

In [49]:
df_forecasts_historic[['venta'] + quantiles]=df_forecasts_historic[['venta'] + quantiles].astype(float)
df_forecasts_historic['month']=pd.to_datetime(df_forecasts_historic['periodo']).dt.to_period('M')
df_forecasts_historic=df_forecasts_historic[['periodo', 'month'] + grouped_columns + ['venta'] + quantiles]

In [50]:
df_forecasts_historic.head(1)

,periodo,month,product_id,venta,mean
0,2017-01-01,2017-01,20001,934.77222,NaN


- Agregamos fuentes adicionales

In [51]:
df_productos = pd.read_csv("C:/Users/usuario/otros/Desktop\MCD/6 Labo III/Forecasting Problem/Datasets/df_products.csv")
df_productos.columns = ["product_id", "cat1", "cat2", "cat3", "brand", "sku_size"]
df_productos["product_id"] = df_productos["product_id"].astype(str)
df_productos.head(5)

,product_id,cat1,cat2,cat3,brand,sku_size
0,20180,HC,ROPA LAVADO,Liquido,LIMPIEX,450
1,20332,HC,ROPA LAVADO,Liquido,LIMPIEX,120
2,20222,HC,ROPA LAVADO,Liquido,LIMPIEX,450
3,20288,HC,ROPA LAVADO,Liquido,LIMPIEX,900
4,20082,HC,ROPA MANCHAS,Ladrillo,LIMPIEX,200


In [52]:
df_forecasts_historic = pd.merge(df_forecasts_historic, df_productos, how = "left", on = "product_id")
df_forecasts_historic.head(5)

,periodo,month,product_id,venta,mean,cat1,cat2,cat3,brand,sku_size
0,2017-01-01,2017-01,20001,934.77222,NaN,HC,ROPA LAVADO,Liquido,ARIEL,3000.0
1,2017-01-01,2017-01,20002,550.15707,NaN,HC,ROPA LAVADO,Liquido,LIMPIEX,3000.0
2,2017-01-01,2017-01,20003,1063.45835,NaN,FOODS,ADEREZOS,Mayonesa,NATURA,475.0
3,2017-01-01,2017-01,20004,555.91614,NaN,FOODS,ADEREZOS,Mayonesa,NATURA,240.0
4,2017-01-01,2017-01,20005,494.27011,NaN,FOODS,ADEREZOS,Mayonesa,NATURA,120.0


- Definición funciones de medición de error

In [53]:
def text_format(val):
    color = 'salmon' if val > 0.3 else 'green'
    return 'color: %s' % color

In [54]:
def format_zero(val):
    color = 'black' if type(val) == str else ('salmon' if val < 0 else 'green')
    return 'color: %s' % color

In [55]:
def forecasts_error_total(df_forecasts, quantile):
    df_forecasts=df_forecasts[df_forecasts['periodo']>=date_min_forecast]
    df_forecasts_grouped = df_forecasts.groupby(['month'] + grouped_columns)[['venta', quantile]].sum().reset_index()
    df_forecasts_grouped['error_absoluto']=abs(df_forecasts_grouped['venta']-df_forecasts_grouped[quantile])
    df_forecasts_grouped =(df_forecasts_grouped.groupby(['month'])['error_absoluto'].sum()
                           /df_forecasts_grouped.groupby(['month'])['venta'].sum()).reset_index(name='error')
    df_forecasts_grouped.replace([np.inf, -np.inf], np.nan, inplace=True)
    return df_forecasts_grouped

In [56]:
def forecasts_error_grouped(df_forecasts, feature, time_column, target_value, quantile):
    df_forecasts=df_forecasts[df_forecasts['periodo']>=date_min_forecast]
    df_forecasts_grouped = df_forecasts.groupby([time_column] + grouped_columns + [feature])[[target_value, quantile]].sum().reset_index()
    df_forecasts_grouped['error_absoluto']=abs(df_forecasts_grouped[target_value]-df_forecasts_grouped[quantile])
    df_forecasts_grouped =(df_forecasts_grouped.groupby([time_column, feature])['error_absoluto'].sum()
                           /df_forecasts_grouped.groupby([time_column, feature])[target_value].sum()).reset_index(name='error')
    df_forecasts_grouped=df_forecasts_grouped.pivot(index=[feature], columns=[time_column], values='error')
    df_forecasts_grouped.replace([np.inf, -np.inf], np.nan, inplace=True)
    df_forecasts_grouped['mean_error']=df_forecasts_grouped.mean(axis=1, skipna=True)
    return df_forecasts_grouped

In [57]:
def forecasts_error_item(df_forecasts, features, time_column, quantile, target_value):
    df_forecasts=df_forecasts[df_forecasts['periodo']>=date_min_forecast]
    df_forecasts_grouped = df_forecasts.groupby([time_column, 'product_id']+features)[[target_value, quantile]].sum().reset_index()
    df_forecasts_grouped['error_absoluto']=abs(df_forecasts_grouped[target_value]-df_forecasts_grouped[quantile])
    df_forecasts_grouped =(df_forecasts_grouped.groupby([time_column, 'product_id'] + features)['error_absoluto'].sum()
                           /df_forecasts_grouped.groupby([time_column, 'product_id']+ features)[target_value].sum()).reset_index(name='error')
    df_forecasts_grouped=df_forecasts_grouped.pivot(index=['product_id'] + features, columns=[time_column], values='error')
    df_forecasts_grouped.replace([np.inf, -np.inf], np.nan, inplace=True)
    df_forecasts_grouped['mean_error']=df_forecasts_grouped.mean(axis=1, skipna=True)
    return df_forecasts_grouped

- Evaluación total general

In [58]:
forecasts_error_total(df_forecasts_historic, "mean").style.applymap(lambda x: text_format(x), ['error'])

,month,error
0,2019-11,0.248667
1,2019-12,0.282917


- Evaluación por categorias

In [59]:
forecasts_error_grouped(df_forecasts_historic, feature='cat1', time_column='month', target_value='venta', quantile='mean').style.applymap(lambda x: text_format(x))

month,2019-11,2019-12,mean_error
cat1,,,
FOODS,0.269497,0.373673,0.321585
HC,0.247416,0.237307,0.242362
PC,0.234862,0.338531,0.286696
REF,3.179160,4.506471,3.842816


In [60]:
forecasts_error_grouped(df_forecasts_historic, feature='cat2', time_column='month', target_value='venta', quantile='mean').style.applymap(lambda x: text_format(x))

month,2019-11,2019-12,mean_error
cat2,,,
ADEREZOS,0.253605,0.321141,0.287373
CABELLO,0.272304,0.427591,0.349948
DENTAL,0.633266,0.276074,0.454670
DEOS,0.185644,0.226405,0.206024
HOGAR,0.230258,0.305668,0.267963
OTROS,0.312276,0.808356,0.560316
PIEL1,0.317740,0.305705,0.311723
PIEL2,0.106213,0.180099,0.143156
PROFESIONAL,0.299091,0.345440,0.322265


- Evaluación por items principales

In [61]:
items_relevantes = ["20001", "20002", "20003", "20004", "20005", "20006", "20007", "20008", "20009", "20010"]

In [62]:
# forecast_mean
forecasts_error_item(df_forecasts_historic[df_forecasts_historic.product_id.isin(items_relevantes)], features=[], time_column='month', target_value='venta', quantile='mean').reset_index().rename_axis(None, axis=1).set_index("product_id").style.applymap(text_format)

,2019-11,2019-12,mean_error
product_id,,,
20001,0.100477,0.004941,0.052709
20002,0.060210,0.210826,0.135518
20003,0.165697,0.042106,0.103901
20004,0.262445,0.246001,0.254223
20005,0.176376,0.084327,0.130351
20006,0.170689,0.121255,0.145972
20007,0.230248,0.127604,0.178926
20008,0.017600,0.943071,0.480335
20009,0.250406,0.094891,0.172648
